# IntelHealth - Colab Training (Per-Agent)

This notebook is designed for **one agent per run**.
Pick `AGENT_NAME`, then run all cells.

All agents use `supervised_finetuning.py` with Qwen ChatML template.
- 4 grader/normalizer agents → Qwen3-0.5B-Instruct (loss on response only)
- diagnosis_generator → Qwen3-1.8B-Instruct (loss on all tokens via `--train_on_inputs`)

In [ ]:
# Clone repo + pull latest
import os
repo_dir = '/content/Intel_Health'
if not os.path.exists(repo_dir):
    !git clone https://github.com/DemonRain7/Intel_Health.git {repo_dir}
else:
    !git -C {repo_dir} pull
%cd {repo_dir}

In [ ]:
# Install dependencies
!pip -q install torch transformers datasets peft accelerate bitsandbytes sentencepiece loguru

In [ ]:
# Mount Google Drive and set persistent paths
from google.colab import drive
drive.mount('/content/drive')

DRIVE_ROOT = '/content/drive/MyDrive/Code_Project/IntelHealth'
SFT_DATA_ROOT = f'{DRIVE_ROOT}/datasets/agent_sft'
ADAPTER_OUTPUT_ROOT = f'{DRIVE_ROOT}/models/adapters'
os.makedirs(SFT_DATA_ROOT, exist_ok=True)
os.makedirs(ADAPTER_OUTPUT_ROOT, exist_ok=True)
print('SFT_DATA_ROOT:', SFT_DATA_ROOT)
print('ADAPTER_OUTPUT_ROOT:', ADAPTER_OUTPUT_ROOT)

## 1) Select Agent (single run)

Supported training agents:
- symptom_normalizer (SFT, 0.5B)
- symptom_quality_grader (SFT, 0.5B)
- rag_relevance_grader (SFT, 0.5B)
- drug_evidence_grader (SFT, 0.5B)
- diagnosis_generator (SFT + train_on_inputs, 1.8B)

In [ ]:
import subprocess, shlex
from pathlib import Path

AGENT_NAME = "symptom_quality_grader"  # <-- change this each run

MODEL_BY_AGENT = {
    "symptom_normalizer":     "Qwen/Qwen3-0.5B-Instruct",
    "symptom_quality_grader": "Qwen/Qwen3-0.5B-Instruct",
    "rag_relevance_grader":   "Qwen/Qwen3-0.5B-Instruct",
    "drug_evidence_grader":   "Qwen/Qwen3-0.5B-Instruct",
    "diagnosis_generator":    "Qwen/Qwen3-1.8B-Instruct",
}

# --train_file_dir expects a DIRECTORY (globs *.jsonl inside)
DATA_DIR_BY_AGENT = {
    "symptom_normalizer":     f"{SFT_DATA_ROOT}/symptom_normalizer",
    "symptom_quality_grader": f"{SFT_DATA_ROOT}/symptom_quality_grader",
    "rag_relevance_grader":   f"{SFT_DATA_ROOT}/rag_relevance_grader",
    "drug_evidence_grader":   f"{SFT_DATA_ROOT}/drug_evidence_grader",
    "diagnosis_generator":    f"{SFT_DATA_ROOT}/diagnosis_generator",
}

OUTPUT_BY_AGENT = {
    "symptom_normalizer":     f"{ADAPTER_OUTPUT_ROOT}/symptom_normalizer",
    "symptom_quality_grader": f"{ADAPTER_OUTPUT_ROOT}/symptom_quality_grader",
    "rag_relevance_grader":   f"{ADAPTER_OUTPUT_ROOT}/rag_relevance_grader",
    "drug_evidence_grader":   f"{ADAPTER_OUTPUT_ROOT}/drug_evidence_grader",
    "diagnosis_generator":    f"{ADAPTER_OUTPUT_ROOT}/diagnosis_generator",
}

assert AGENT_NAME in MODEL_BY_AGENT, f"Unknown agent: {AGENT_NAME}"

MODEL_NAME = MODEL_BY_AGENT[AGENT_NAME]
TRAIN_DIR  = DATA_DIR_BY_AGENT[AGENT_NAME]
OUTPUT_DIR = OUTPUT_BY_AGENT[AGENT_NAME]

print(f"AGENT_NAME: {AGENT_NAME}")
print(f"MODEL_NAME: {MODEL_NAME}")
print(f"TRAIN_DIR:  {TRAIN_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

## 2) Build command

All agents use `supervised_finetuning.py` with Qwen ChatML template.
- `diagnosis_generator` adds `--train_on_inputs` (loss on all tokens, DAPT-like behavior)
- Others only compute loss on model response tokens

In [ ]:
SFT_SCRIPT = Path("training/supervised_finetuning.py")

# --- Hyperparameters ---
EPOCHS     = 3
LR         = "2e-4"
BATCH      = "2"
GRAD_ACC   = "8"           # effective batch = 2 * 8 = 16
LORA_R     = "64"
LORA_ALPHA = "128"         # scaling = alpha/rank = 2.0
LORA_DROPOUT = "0.05"
MAX_LEN    = "512"         # max sequence length

cmd = [
    "python", str(SFT_SCRIPT),
    "--model_name_or_path",        MODEL_NAME,
    "--tokenizer_name_or_path",    MODEL_NAME,
    "--train_file_dir",            TRAIN_DIR,
    "--output_dir",                OUTPUT_DIR,
    "--template_name",             "qwen",
    "--do_train",
    "--fp16",
    "--gradient_checkpointing",
    "--per_device_train_batch_size", BATCH,
    "--gradient_accumulation_steps", GRAD_ACC,
    "--num_train_epochs",          str(1 if AGENT_NAME == "diagnosis_generator" else EPOCHS),
    "--learning_rate",             LR,
    "--lora_rank",                 LORA_R,
    "--lora_alpha",                LORA_ALPHA,
    "--lora_dropout",              LORA_DROPOUT,
    "--model_max_length",          MAX_LEN,
    "--logging_steps",             "10",
    "--save_strategy",             "epoch",
    "--overwrite_output_dir",
]

# diagnosis_generator: train on ALL tokens (including the question)
if AGENT_NAME == "diagnosis_generator":
    cmd.append("--train_on_inputs")

print("Command:")
print(" ".join(shlex.quote(x) for x in cmd))

In [ ]:
# Run training
subprocess.run(cmd, check=True)

## 3) Next agent

After this run finishes:
1. Change `AGENT_NAME` in cell 1
2. **Runtime → Restart runtime** (free GPU memory)
3. Run all cells again

Trained LoRA adapters are saved to Google Drive: `models/adapters/<agent>/`

### Recommended training order
1. `symptom_normalizer` (0.5B, fast)
2. `symptom_quality_grader` (0.5B, fast)
3. `rag_relevance_grader` (0.5B, fast)
4. `drug_evidence_grader` (0.5B, fast)
5. `diagnosis_generator` (1.8B, slower)